In [1]:
import numpy as np
import pandas as pd
import os
from datetime import date
import sqlalchemy
import pymysql
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [122]:
file_name = '..\..\..\dataExport\LOGdata\log_2022_4_20.csv'
log_rawDF = pd.read_csv(file_name, sep='*')

In [123]:
feed_temp_ins4 = log_rawDF.query('Time >= 1635246000 and Time <= 1641567595 and Unit =="FPC24" and Type == 2 and Parameter == "Reactor"')

In [124]:
programID_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('Recipe', regex=False, case=False, na=False)]

In [128]:
programID_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350408,2021-11-2 8:37:25,1635838645,243296,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.0857142857142857
350510,2021-11-2 10:57:24,1635847044,749185,2,FPC24,Reactor,Recipe of Complete 3_5de half started in FFU_T10 (kg),NaN,0.514285714285714
350657,2021-11-2 11:33:29,1635849209,742078,2,FPC24,Reactor,Recipe of FeedWaterAcid started in FFU_T10 (kg),NaN,0.257142857142857
352123,2021-11-3 8:33:30,1635924810,738753,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.114285714285714
352176,2021-11-3 10:17:39,1635931059,739134,2,FPC24,Reactor,Recipe of Complete 3_5de half started in FFU_T10 (kg),NaN,0.685714285714286


In [130]:
new_DF = log_rawDF.copy()

In [131]:
new_DF_ins4 = new_DF.query('Time >= 1635246000 and Time <= 1641567595')

In [132]:
new_ins4 = new_DF_ins4[new_DF_ins4['Message'].str.contains('FPC24', regex=False, case=False, na=False)]
#programID_DF = programID_DF[programID_DF['Value2'].str.contains('FPC24', regex=False, case=False, na=False)]

In [134]:
new_ins4.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350421,2021-11-2 8:40:21,1635838821,352002,2,FFU,T10,"Dose started for Si_1_5de to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.06067</td><td width=""12.5%"">level</td><td width=""12.5%"">0.04036</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.08571,0.00000
350468,2021-11-2 10:49:20,1635846560,869377,2,FFU,T10,"Dose ended at low level for Si_1_5de to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.09083</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.03131</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.09083,0.03920
350625,2021-11-2 11:14:52,1635848092,351563,2,FFU,T10,"Dose started for Complete 3_5de half to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.50102</td><td width=""12.5%"">level</td><td width=""12.5%"">0.52379</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.51429,0.00000
350629,2021-11-2 11:19:47,1635848387,866793,2,FFU,T10,"Dose ended at low level for Complete 3_5de half to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.07249</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.02936</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.50102,0.49444
350664,2021-11-2 11:34:48,1635849288,851779,2,FFU,T10,"Dose started for FeedWaterAcid to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.25719</td><td width=""12.5%"">level</td><td width=""12.5%"">0.28841</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.25714,0.00000


In [141]:
new_ins4.tail()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
446599,2021-12-29 16:23:29,1640791409,196243,2,FFU,T10,"Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.04420</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.04361</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.22822,0.21332
446622,2021-12-29 16:30:7,1640791807,667000,2,FFU,T10,"Dose started for NaOH_0.1M_rinse to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.91182</td><td width=""12.5%"">level</td><td width=""12.5%"">0.94773</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.91429,0.00000
446634,2021-12-29 16:40:9,1640792409,187728,2,FFU,T10,"Dose ended at low level for NaOH_0.1M_rinse to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.05251</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.04559</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.91182,0.90214
447338,2021-12-30 9:38:25,1640853505,670297,2,FFU,T10,"Dose started for FeedWater to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.10228</td><td width=""12.5%"">level</td><td width=""12.5%"">1.15110</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",1.14286,0.00000
447351,2021-12-30 9:49:54,1640854194,205742,2,FFU,T10,"Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.08463</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.04373</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",1.14025,1.14535


In [138]:
no_html_DF =  new_ins4.copy()

In [139]:
#no_html_DF['Message'] = no_html_DF['Message'].str.replace(r'<[^<>]*>', '', regex=True)
no_html_DF['Message'] = no_html_DF['Message'].str.replace(r'<.*?>.*?</.*?>', '', regex=True)

In [140]:
no_html_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350421,2021-11-2 8:40:21,1635838821,352002,2,FFU,T10,Dose started for Si_1_5de to Tank = FPC24_Reactor (target dose and current real dose),0.08571,0.00000
350468,2021-11-2 10:49:20,1635846560,869377,2,FFU,T10,Dose ended at low level for Si_1_5de to Tank = FPC24_Reactor (DB total dose and current real dose),0.09083,0.03920
350625,2021-11-2 11:14:52,1635848092,351563,2,FFU,T10,Dose started for Complete 3_5de half to Tank = FPC24_Reactor (target dose and current real dose),0.51429,0.00000
350629,2021-11-2 11:19:47,1635848387,866793,2,FFU,T10,Dose ended at low level for Complete 3_5de half to Tank = FPC24_Reactor (DB total dose and current real dose),0.50102,0.49444
350664,2021-11-2 11:34:48,1635849288,851779,2,FFU,T10,Dose started for FeedWaterAcid to Tank = FPC24_Reactor (target dose and current real dose),0.25714,0.00000


In [142]:
no_html_DF.tail()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
446599,2021-12-29 16:23:29,1640791409,196243,2,FFU,T10,Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose),0.22822,0.21332
446622,2021-12-29 16:30:7,1640791807,667000,2,FFU,T10,Dose started for NaOH_0.1M_rinse to Tank = FPC24_Reactor (target dose and current real dose),0.91429,0.00000
446634,2021-12-29 16:40:9,1640792409,187728,2,FFU,T10,Dose ended at low level for NaOH_0.1M_rinse to Tank = FPC24_Reactor (DB total dose and current real dose),0.91182,0.90214
447338,2021-12-30 9:38:25,1640853505,670297,2,FFU,T10,Dose started for FeedWater to Tank = FPC24_Reactor (target dose and current real dose),1.14286,0.00000
447351,2021-12-30 9:49:54,1640854194,205742,2,FFU,T10,Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose),1.14025,1.14535
